# Classification: K-nearest-neighbours

Si vous vous rapplez du chapitre 2, le Machine Learning peut être utilisé pour deux tâches: la régression et la classification. Nous avons introduit la régression dans le chapitre précédent, il est donc temps de présenter la classification. 
L'approche la plus simple et intuitive est de partir du principe que les points de données de même catégorie se ressemblent. En partant de cette hypothèse, nous pouvons créer un modèle qui va commparer le point que nous souhaitons prédire ($x_{new}$) avec ceux qui sont déjà connu et aggréger les plus proches. 
C'est exactement ce que fait K-nearest-neighbours ("k voisins les plus proches"). Il va d'abord stocker les données d'entrainment avec leurs labels respectifs. Ensuite, lors de la prédiction, il va trouver les $k$ voisins les plus proches et aggréger leur labels respectifs. 

## 1. K-nearest-neighbours

Il faudra résoudre plusieurs problèmes avant de pouvoir utiliser kNN. En effet, il faut déterminer le nombre de voisins ($k$) à prendre en compte, la façon de calculer les distances ainsi que la façon d'aggréger les voisins.

### 1.1 Mesure de Distance
Nous pouvons utiliser n'importe quelle formule tel que l'Euclidienne (l2-norm), Manahttan (l1-norm) et plus. Le mesure la plus simple est la distance euclidienne qui calcule la distance dans l'espace cartésien.

$$distance(x_{new},x_n)=\sqrt{\sum_{a=1}^A(x_{new,a}-x_{n,a})^2}$$
$A$ représente tous les attributs.

Par facilité nous pouvons omettre la racine carrée et réecrire la formule en utilisant des vecteurs:

\begin{align}
    distance(x_{new},x_n)^2 &= \sum_{a=1}^A(x_{new,a}-x_{n,a})^2 \\
    &= \sum_{a=1}^A(x_{new,a}-x_{n,a})^T(x_{new,a}-x_{a,d}) \\
    &= (x_{new}-x_n)^T(x_{new}-x_n)
\end{align}

Cette formulation est la plus répendue et est pratique car les ordinateurs sont particulièrement performant dans le calcul vectoriel.

Il est aussi possible de calculer la distance dans un autre espace grâce aux kernels. Cependant, cette méthode ne sera pas expliqué dans ce module. 

### 1.2 Nombre de voisins

Maintenant que nous savons comment la distance sera mesurée il faut savoir combien de voisins nous devons prendre en compte. Vous pouvez voir dans l'image que le nombre de k a un impact significatif sur la qualité et la forme du modèle. Par exemple $k=1$ crée des poches de bleu dans le rouge, ainsi des échantillons entourée de points rouge mais proche du point bleu seront bleu. Dans ce cas on dit que le modèle overfit car il repose trop sur les données d'entrainement. Utiliser $k=5$ semble retirer les poches. Prendre 10 voisins ou plus semble lisser la délimitation entre les classes.

**Une image est présente pour montrer l'impact du nombre de voisins**
<img src="choix_des_k.png" alt="drawing" width="750"/>

Il est dont primordial de déterminer le nombre obtimal de voision à prendre en compte. Le plus efficace et simple, et d'utiliser les données d'entrainement pour trouver ce nombre. Nous pouvons déterminer une liste de nombres des voisins, et d'entrainer chaque version avec l'ensemble d'entrainement et de l'évaluer sur l'ensemble de validation. La version du modèle la plus performante est celle que nous garderons. 

Un autre facteur déterminant est la balance des données. Si notre dataset d'entrainement n'est pas balancé et que nous avons 10 points qui sont de class 1 et 90 de class 2, utiliser un k > 10 va toujours prédire 2. 

### 1.3  Agrégation des résultats
La dernière chose à déterminer est la façon dont on va aggréger les voisins. KNN est utilisé plus souvent dans les tâches de classification mais ce modèle peut aussi être utilisé pour de la régression.

#### 1.3.1 Classification

La plus simple est d'attribuer la classe qui est la plus présente dans les voisins qui ont été selectionné. Dans certain cas, on voudra prendre la distance des voisins en compte. Il est possible que notre point soit collé a deux points d'une classe mais qu'il ait 3 voisins assez distant. Si nous utilisons la méthode précente, le point sera classifié comme appartenant à la même class que les trois points distant. Ce problème peut-être résolu en prenant la distance en compte. Pour ce faire, on attribue un poids à chaque vote, tel que l'inverse de la distance, les deux points proche auront l'avantage et notre nouveau point sera correctement classifié. 

#### 1.3.2 Régression
Dans le cas de régression, nous prendrons la moyenne de chaque voisins. Dans ce cas nous pouvons aussi prendre la distance en compte. 

### 2. Point Important
Nous savons que dans le cadre de la classification, KNN part du principe que les échantillons d'une même catégorie sont proche l'un de l'autre et que, idéalement, les données d'entrainement sont balancée. Un autre point important à prendre en compte est ce qu'on appel "la malédiction de la dimensionnalité". Le volume d'une espace de données augumente exponentielement avec le nombre d'attributs, ainsi nous avons besoin d'une quantité exponentielle d'échantillons, sinon nous aurons des espaces quasi vide et la distance entre chaque échantillon augumentera. KNN évalue la distance entre un point et un autre en fonction de la distance dans chaque attribut ($(x_{new,a}-x_{n,a})^2$), il sera donc de plus en plus difficile pour KNN de trouver les voisins correctement. 

## 3. À retenir

- Lors de la classification, Knn part du principe que les échantillons de la même catégorie sont proches l'un de l'autre.
- KNN est un modèle assez simple mais qui est modulable afin de correspondre le plus possible à notre tâche.
- Il faut faire attention à la balance de l'ensemble d'entrainement. 
- Ce modèle peut-être utilisé pour la classification ainsi que la régression. 


## 4. Implémenter Knn

In [33]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


'''
-- Création de l'ensemble de donnée --
Notez que j'utilise une distribution normal ce qui permet d'avoir tous les points d'une même class relativement proche.
De plus, je génère un ensemble balancé
'''

elements = 50
random_state = 42
K = 5

def generate_data(elements, random_state):
  rng=np.random.RandomState(random_state)

  classes_x = rng.normal(0, 0.55, (2,elements))
  classes_y = rng.normal(1, 0.55, (2,elements))
  labels = np.concatenate((np.zeros(elements), np.ones(elements)))
  dset = np.zeros((2,elements*2))

  dset[0] = np.concatenate((classes_x[0], classes_y[0]))
  dset[1] = np.concatenate((classes_x[1], classes_y[1]))

  return dset, labels

'''
-- Implémentation du modèle --
'''

class KNN:
    def __init__(self, K):
        self.K = K
    

    def fit(self, data, labels):
        self.data = data
        self.labels = labels
    
    
    def predict(self, data_point):
        preds = []

        for dp in data_point:
          distances = []
          
          for point in self.data:
              distances.append(self.distance(dp, point)) 
              
          sorted_distances = np.argsort(distances)

          preds.append(np.bincount(self.labels[sorted_distances[:self.K]].astype(int)).argmax())
        
        return preds
        #return np.bincount(self.labels[sorted_distances[:self.K]].astype(int)).argmax()
        

    def distance(self, x_new, x):
        return np.power(np.dot(x_new-x, x_new-x), 2)
    

'''
-- Utilisation --
'''

# On génère les données d'entrainement et on "fit" le modèle
train_set, train_labels = generate_data(elements, random_state)

model = KNN(K)
model.fit(train_set.T, train_labels)

# Ensuite on le test en générant le set de test 
elements = 10
test_set, test_labels = generate_data(elements, random_state)

preds = model.predict(test_set.T)

print(f"Predit: {preds}")
print(f"Réel:   {test_labels}")

'''
Préparation du plot en utilisant plotly
'''

predicted = pd.DataFrame({"x": test_set[0] , "y": test_set[1], "labels": preds})
df = pd.DataFrame({"x": train_set[0] , "y": train_set[1], "labels": train_labels})

df["labels"] = df["labels"].astype(str)
predicted["labels"].loc[predicted["labels"] != test_labels] = "Faux"
predicted["labels"] = predicted["labels"].map({0:"Predit 0", 1:"Predit 1", "Faux":"Faux"})

mesh_size = .02
margin = 0.25

# Create a mesh grid on which we will run our model
x_min, x_max = train_set[0].min() - margin, train_set[0].max() + margin
y_min, y_max = train_set[1].min() - margin, train_set[1].max() + margin
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)
xx, yy = np.meshgrid(xrange, yrange)

# Create classifier, run predictions on grid
Z = np.array(model.predict(np.c_[xx.ravel(), yy.ravel()])).astype(str)
Z = Z.reshape(xx.shape)

Predit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]
Réel:   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<!--
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier

h = .02
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

random_state = 42
rng=np.random.RandomState(random_state )

x1 = rng.normal(0, 0.55, 50)
y1 = rng.normal(0, 0.55, 50)

x2 = rng.normal(1, 0.55, 50)
y2 = rng.normal(1, 0.55, 50)

x = np.concatenate((x1, x2))
y = np.concatenate((y1, y2))
labels = np.concatenate((np.full(len(x1), 0), np.full(len(x2), 1)))

f, axis = plt.subplots(1, 4, sharey=True, figsize=(15,15))

line = 0

for ind, k in enumerate([1, 5, 10, 15]):
    
    if ind > 3:
        ind -= 4
        line = 1
        
    X = np.transpose(np.array([x,y]))
                        
    kNN = KNeighborsClassifier(k)
    kNN.fit(X, labels)

    np.transpose(X)

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = kNN.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)

    #plt.figure()
    axis[ind].pcolormesh(xx, yy, Z, cmap=cmap_light)
    # Plot also the training points
    axis[ind].scatter(X[:,0], X[:,1], c=labels, cmap=cmap_bold)
    axis[ind].set_xlim(xx.min(), xx.max())
    axis[ind].set_ylim(yy.min(), yy.max())
    axis[ind].set(aspect=1)
    axis[ind].set_title(f"k={k}")
    
plt.tight_layout()
plt.savefig("choix_des_k", bbox_inches='tight', pad_inches=0)
plt.show()
-->

In [34]:
# Plot the figures
fig0 = px.scatter(df, x="x", y="y", color="labels", color_discrete_sequence=["rgb(2,48,71)", "rgb(255,158,2)"])
fig1 = px.scatter(predicted, x="x", y="y", color="labels", 
                  symbol_sequence=["square"], color_discrete_sequence=["rgb(2,48,71)", "rgb(255,158,2)", "rgb(255,255,255)"])

fig1.update_traces(marker=dict(size=5,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

color_scale = [[0, "rgb(255, 255, 255)"], [1, "rgb(122, 122, 122)"]]

fig2 = go.Figure(data=[
    go.Contour(
        x=xrange,
        y=yrange,
        z=Z,
        colorscale=color_scale,
        showscale=False
    ),

])

fig3 = go.Figure(data=fig0.data + fig1.data + fig2.data)
fig3.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig3.update_xaxes(
    range=[x_min, x_max], 
    constrain="domain",
)

fig3.show()

In [18]:
print(Z)

[['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ['0' '0' '0' ... '0' '0' '0']
 ...
 ['1' '1' '1' ... '1' '1' '1']
 ['1' '1' '1' ... '1' '1' '1']
 ['1' '1' '1' ... '1' '1' '1']]
